# Using Numba for GPUs

*Vasileios Karakasis, **Theofilos Manitaras**, CSCS*

You may use Numba to generate GPU code from high-level Python functions. Both CUDA and ROCm GPUs (NVIDIA and AMD, respectively) are supported, but we are going to focus only on the CUDA capabilities of Numba.

There is an almost one-to-one mapping between the Numba high-level Python abstractions and the different CUDA constructs. This practically means that, as a programmer, you need to take care explicitly of the host/device memory management and you need to be aware of the CUDA programming model.

This demo will teach you to write your first CUDA kernels using Numba. It will cover the basic CUDA programming principles, but it should be enough to kick start you in GPU programming. More specifically, we will cover the following topics:

- Writing a GPU kernel
- Moving data to/from the GPU.
- Spawning a GPU kernel
- Profiling a GPU kernel
- Optimizing memory accesses
- Making use of the shared memory

We will not cover CUDA streams.

## Verify that Numba sees the GPU and understands CUDA

First thing is to check if Numba can detect the GPU. You can achieve this by running the `numba` executable that comes with Numba's installation:

In [2]:
!numba -s

System info:
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
--------------------------------------------------------------------------------
__Time Stamp__
Report started (local time)                   : 2022-06-20 13:52:55.653965
UTC start time                                : 2022-06-20 11:52:55.653969
Running time (s)                              : 1.036071

__Hardware Information__
Machine                                       : x86_64
CPU Name                                      : skylake
CPU Count                                     : 12
Number of accessible CPUs                     : 12
List of accessible CPUs cores                 : 0 1 2 3 4 5 6 7 8 9 10 11
CFS Restrictions (CPUs worth of runtime)      : None

CPU Features                                  : 64bit adx aes avx avx2 bmi bmi2
                                                clflushopt cmov cx16 cx8 f16c fma
                                                fsgs

If this command does not work out of the box for your Numba installation, you may have to set the `CUDA_HOME` environment variable to point to your CUDA installation.

## Writing the first kernel

Here is a vector addition kernel that takes two vectors as input and writes the sum in a third vector:

In [3]:
import numba.cuda as cuda


@cuda.jit
def _vecadd_cuda(z, x, y):
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]


Pretty simple, right? Let's explain each line of this code.

The `@cuda.jit` decorator will compile the following function into a CUDA kernel at runtime. We will see the cost of it later on.

A CUDA kernel in Numba is a Python function that does *not* return a value. This is in accordance with CUDA, where kernel functions are declared `void`. The arguments of the function can be either Numpy arrays or scalars of a Numba recognized type.

CUDA kernels specify the work to be done by a single GPU thread. Due to the massive hardware parallelism available on the GPU, a single GPU thread gets only a tiny portion of work, in this case, the sumation of a single element in the vectors. Since each element is independent from each other, we can safely create as many threads as the elements of the target vectors and spawn them (we will see how later).

Threads on the GPU as organized in groups, called CUDA blocks. There are constraints on the maximum number of threads a block can contain. For the P100 GPUs on Daint, this is 1024. This means that you will need multiple blocks to calculate the sum of large vectors. The blocks that comprise a kernel form the *grid*. Threads inside a block are numbered sequentially and the blocks inside the grid are numbered, too. The figure below shows the arrangement of threads for the vector addition example.

![Arrangement of threads in CUDA blocks](figs/cuda-blocks.png)

In order to obtain the i-th element of the vector given a block size $B$, you would have to calculate the following:

\begin{equation}
i = i_{b}B + i_{t}
\end{equation}

where $i_{b}$ is the block index and $i_{t}$ is the thread index. CUDA provides this information and Numba makes it available, so that the above statement would be written as follows:

```python
i = cuda.blockIdx.x*cuda.blockDim.x + threadIdx.x
```

Blocks and grids can be three dimensional, thus the `.x` attribute in all of these variables (the other dimensions are accessed through the `.y` and `.z` attributes).

Since obtaining the absolute position of a thread in a CUDA is quite common operation, Numba offers the convenience function `grid()`, which essentially does automatically the above calculation:

```python
i = cuda.grid(1)
```

The argument passed to the `grid()` function is the number of dimensions of the grid.

The next three statements in the code simply check that we don't overrun the arrays in case that their dimension is not a multiple of the block size. In this case, some threads of the last block will remain idle:

```python
    N = x.shape[0]
    if i >= N:
        return
```

> The threads inside a block are run in batches of 32 at once, called *warps*. All the threads of the warp execute the same instruction. If the program control flow diverges for some of the threads of the warp, e.g., due to an `if` condition, the branches will be executed sequentially by disabling the non participating threads. This condition is called *warp divergence* and may incur a performance penalty.

Finally, the `z[i] = x[i] + y[i]` computes the actual sum.


## Preparing the data for the GPU

All Numba CUDA kernels operate on data residing on the GPU. That means that the `x`, `y` and `z` arrays must be  transferred from the host (CPU) to the device (accelerator) before calling the CUDA kernel.

Let's create first two vectors on the host:

In [4]:
import numpy as np

rng = np.random.default_rng()

N = 1024
x = rng.random(N)
y = rng.random(N)

Let's also compute our reference result:

In [5]:
z_ref = x + y

Now it's time to transfer the vectors to the GPU:

In [6]:
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

The `d_x` and `d_y` are Numpy *array-like* objects that have their data mapped in the GPU memory:

In [7]:
d_x

For the `z` array, we don't need to create it on the host and copy it, since it is essentially an output only array. We can simply allocate it directly on the GPU and copy it out when the kernel finishes.

In [8]:
d_z = cuda.device_array_like(x)

This will simply allocate an array like `x`, i.e., with the same element type, shape and order, on the GPU.

Data is ready, now it's time to call the kernel!

## Invoking a CUDA kernel

When invoking a CUDA kernel you have to specify the block size to use and the corresponding grid size. Picking the right size of block is not always straightforward, but usually values between 64 and 256 are good enough. 

> The block size has a direct effect on the *occupancy* of each GPU SM, i.e., how much the actual hardware threads of the SM are utilized, and as a result to the occupancy of the whole GPU. Performance-wise, though, it does not have such a big impact. Nvidia provides a nice tool for calculating the occupancy of the GPU, that you can find [here](https://docs.nvidia.com/cuda/cuda-occupancy-calculator/CUDA_Occupancy_Calculator.xls).

For our kernel, we will select a block size of 128 threads.

In [9]:
block_size = 128

Having decided the block size we need to set up the grid:

In [10]:
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

Now we are ready to call the kernel!

In [11]:
_vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

/home/teojgo/.local/lib/python3.9/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (8) < 2 * SM count (26) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Both the block and the grid could have been two-dimensional (not in this example), in which case you would just define them as tuples.

## Copying back the results

As mentioned before, kernels operate on GPU data only. We need a way to transfer back to the host the result, which is the `d_z` array. Here's how:

In [12]:
res = d_z.copy_to_host()

Let's validate the result though to make sure that everything has worked fine:

In [13]:
assert np.allclose(z_ref, res)

## Putting it all together

For completeness and easy reference, here is the whole vector addition example:

In [14]:
import numba.cuda as cuda
import numpy as np


@cuda.jit
def _vecadd_cuda(z, x, y):
    '''The CUDA kernel'''
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]


# Set up the host vectors
N = 1000
x = rng.random(N)
y = rng.random(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
_vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

/home/teojgo/.local/lib/python3.9/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (8) < 2 * SM count (26) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Exercise

> 1. Make the array sufficiently large and time the Numpy version of the sum `x + y`.
> 2. Now time the call to the CUDA kernel with `%timeit -n1 -r1`. What do you see?
> 3. Try timing the CUDA kernels with `%timeit -n1 -r2`. What is happening?